In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the fhuA gene
model0['lb_list']['EX_fecrm_e ']=0
model0['lb_list']['EX_fecrm_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 20, 19, 17, 11,  3,  9, 14,  4,  8,  9,  8, 18, 18, 16, 15,  8,
        4, 14,  6, 10, 20, 16, 11, 18, 20, 14, 10, 15,  7]), 1: array([10,  9,  7,  9, 10, 17,  8,  7,  8,  1, 19, 10, 20,  3,  3, 12, 11,
       17,  3, 16, 12, 11,  9,  4,  6,  8,  8,  2, 15,  7])}
1
strain_number:  0 14.166666666666666
strain_various:  0 6.542595475463506
strain_number:  1 9.4
strain_various:  1 4.930179171862486
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 20, 19, 17, 11,  3,  9, 14,  4,  8,  9,  8, 18, 18, 16, 15,  8,
        4, 14,  6, 10, 20, 16, 11, 18, 20, 14, 10, 15,  7]), 1: array([10,  9,  7,  9, 10, 17,  8,  7,  8,  1, 19, 10, 20,  3,  3, 12, 11,
       17,  3, 16, 12, 11,  9,  4,  6,  8,  8,  2, 15,  7])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.678486251

glc__D_e_1:  -26.019446929513233
glc__D_e_1:  -17.851153744427428
glc__D_e_1:  -22.904205268644837
glc__D_e_1:  -29.808593844845632
glc__D_e_1:  -18.288317053693433
glc__D_e_1:  -12.43638877396894
glc__D_e_1:  -15.102490232966062
glc__D_e_1:  -21.93568994909488
glc__D_e_1:  -19.263632625328352
glc__D_e_1:  -21.351160249460154
glc__D_e_1:  -19.312975541601624
glc__D_e_1:  -11.236481412968994
glc__D_e_1:  -0.33267905464645614
glc__D_e_1:  -4.853894124188908
glc__D_e_1:  -31.51738970710893
glc__D_e_1:  -0.7832984789663371
glc__D_e_1:  -0.514240708819669
glc__D_e_1:  -24.015066049803742
glc__D_e_1:  -39.36719676789116
glc__D_e_1:  -0.3283745091713861
glc__D_e_1:  -23.434758143706137
glc__D_e_1:  -27.66303203299226
glc__D_e_1:  -13.74702212931744
glc__D_e_1:  -22.769479965139773
glc__D_e_1:  -7.924639643079989
glc__D_e_1:  -14.952804551174024
glc__D_e_1:  -10.620339836360747
glc__D_e_1:  -16.00459218179327
glc__D_e_1:  -29.258431189513104
glc__D_e_1:  -22.985194438986493
glc__D_e_1:  -23.95

glc__D_e_1:  -1.66433040448632
glc__D_e_1:  -0.5716549216762434
glc__D_e_1:  -0.5677845821353386
glc__D_e_1:  -1.5584049292429634
strain_number:  0 3.2333333333333334
strain_various:  0 1.764149149653232
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 616.852428273481, 304.5616390188759, 337.8182549906235, 415.18976004551143, 399.802045087529, 422.25166305787434, 442.91821695038766, 493.23105808033876, 380.45771883773693, 337.4182253789339, 297.66779149445125, 447.76682204566015, 375.73979534955674, 220.49736625928512, 250.20481929479303, 185.83724406398554, 201.47777113237638, 204.38609307359306]
slip_r:  204.38609307359306
20
glc__D_e_1:  -0.5048164795158206
glc__D_e_1:  -0.4509664660955973
glc__D_e_1:  -0.49156621641977805
glc__D_e_1:  -0.9431235402927316
glc__D_e_1:  -1.2547487478858033
glc__D_e_1:  -2.6051783766932592
glc__D_e_1:  -1.6353263649094176
glc__D_e_1:  -1.7153544103043472
glc__D_e_1:  -1.6526746972360788
glc__D_e_1:  -1.4595944802

In [7]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 5, 17,  4,  2, 19,  2, 12,  3, 13,  8, 20, 18, 19, 14,  2, 20,  6,
       13,  5,  2,  8,  2,  2, 18, 12, 18, 20, 17, 14,  6]), 1: array([11, 10,  5,  1,  6, 14, 19,  2,  5,  6, 17, 16,  1, 13, 17,  7, 20,
       16, 19,  6,  6, 10, 10, 20, 18,  8,  6, 10, 16,  9])}
1
strain_number:  0 12.4
strain_various:  0 8.044045416745648
strain_number:  1 10.8
strain_various:  1 5.79885046079537
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 5, 17,  4,  2, 19,  2, 12,  3, 13,  8, 20, 18, 19, 14,  2, 20,  6,
       13,  5,  2,  8,  2,  2, 18, 12, 18, 20, 17, 14,  6]), 1: array([11, 10,  5,  1,  6, 14, 19,  2,  5,  6, 17, 16,  1, 13, 17,  7, 20,
       16, 19,  6,  6, 10, 10, 20, 18,  8,  6, 10, 16,  9])}
_2:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 5: [0.6459265309330249, 0.285

glc__D_e_1:  -17.027224970986037
glc__D_e_1:  -10.308232065816725
glc__D_e_1:  -19.351510362843083
glc__D_e_1:  -0.08813489711907252
glc__D_e_1:  -7.443339844559997
glc__D_e_1:  -20.858449896717136
glc__D_e_1:  -0.7305139014135964
glc__D_e_1:  -0.921955054363925
glc__D_e_1:  -9.799394471749292
strain_number:  0 43.166666666666664
strain_various:  0 22.28016058190684
strain_number:  1 1.2333333333333334
strain_various:  1 1.7451520150277897
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 770.6062834868949, 296.93159253027676, 295.1544289044289, 345.9263638322463, 384.23938736701894, 356.81483088709325, 381.7060742504218, 411.0114926258232, 398.7165207821816, 451.5142176613566, 377.1399720455771, 379.58613212465207, 398.85084342671166]
slip_r:  398.85084342671166
15
glc__D_e_1:  -17.3849321209405
glc__D_e_1:  -25.651205273726926
glc__D_e_1:  -18.032916852876326
glc__D_e_1:  -24.225080445131624
glc__D_e_1:  -28.12615647477632
glc__D_e_1:  -6.682082426559863
glc__D_e_1:  -13.774969463140865
glc

glc__D_e_1:  -4.24045493827343
glc__D_e_1:  -7.569704268623565
glc__D_e_1:  -5.245497332324912
glc__D_e_1:  -5.702952813366343
glc__D_e_1:  -4.892574931061122
glc__D_e_1:  -1.909273406930337
glc__D_e_1:  -1.717372407722614
glc__D_e_1:  -0.08631624804885307
glc__D_e_1:  -0.6304069967224666
glc__D_e_1:  -0.5947857701672712
glc__D_e_1:  -0.8007316828529398
glc__D_e_1:  -0.18526627931228806
glc__D_e_1:  -0.7561672321879357
glc__D_e_1:  -0.2749659632293494
glc__D_e_1:  -0.8351245417782815
glc__D_e_1:  -1.8518907973169847
glc__D_e_1:  -3.1921685958559944
glc__D_e_1:  -3.6643128173447144
glc__D_e_1:  -6.766227207555545
glc__D_e_1:  -2.271859464305332
glc__D_e_1:  -0.9336694397170983
glc__D_e_1:  -0.43724697962615966
glc__D_e_1:  -0.9331093224020461
glc__D_e_1:  -0.22269082624229775
glc__D_e_1:  -0.11365667877246738
glc__D_e_1:  -0.08647179783017322
glc__D_e_1:  -0.4599661791616223
strain_number:  0 3.1666666666666665
strain_various:  0 2.745703714694812
strain_number:  1 0.0
strain_various:  

In [8]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8,  5, 18,  3,  4, 11,  8, 19,  6, 19,  2,  9, 11, 18, 14,  1, 15,
       10, 17, 19,  1,  4,  5, 17, 13, 18, 20, 12,  8, 18]), 1: array([14,  8,  9,  3, 11,  7, 12, 13,  7, 18, 14, 20, 17, 20, 11, 12, 14,
       17, 16,  7, 14, 15,  3, 15,  6,  8,  7, 12,  7,  5])}
1
strain_number:  0 12.866666666666667
strain_various:  0 7.347259504211228
strain_number:  1 11.4
strain_various:  1 4.722993401082269
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8,  5, 18,  3,  4, 11,  8, 19,  6, 19,  2,  9, 11, 18, 14,  1, 15,
       10, 17, 19,  1,  4,  5, 17, 13, 18, 20, 12,  8, 18]), 1: array([14,  8,  9,  3, 11,  7, 12, 13,  7, 18, 14, 20, 17, 20, 11, 12, 14,
       17, 16,  7, 14, 15,  3, 15,  6,  8,  7, 12,  7,  5])}
_2:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.5207337757120

glc__D_e_1:  -8.611596259375228
glc__D_e_1:  -24.56800004452176
glc__D_e_1:  -2.7217884930197673
glc__D_e_1:  -1.1828544917670396
glc__D_e_1:  -4.0487850916498775
glc__D_e_1:  -25.81860467607515
glc__D_e_1:  -3.788667348373643
glc__D_e_1:  -1.0087670414752554
glc__D_e_1:  -3.35362376829356
glc__D_e_1:  -16.13532307686976
glc__D_e_1:  -2.078206666889942
glc__D_e_1:  -1.6251913259913837
glc__D_e_1:  -14.391525514100344
glc__D_e_1:  -31.544213729180814
glc__D_e_1:  -3.613706685420688
glc__D_e_1:  -1.1527681896761415
glc__D_e_1:  -10.271585266333373
glc__D_e_1:  -16.579823501300886
glc__D_e_1:  -1.1853589944560472
glc__D_e_1:  -46.57975522881667
glc__D_e_1:  -31.937785448835587
glc__D_e_1:  -0.4801823867133983
glc__D_e_1:  -0.2962014455229154
glc__D_e_1:  -15.146831125890007
glc__D_e_1:  -38.29069768845525
glc__D_e_1:  -0.7487991340749325
glc__D_e_1:  -15.305341339040794
glc__D_e_1:  -34.73777832383847
glc__D_e_1:  -1.090427340219236
glc__D_e_1:  -8.404581947444795
glc__D_e_1:  -23.3311441

glc__D_e_1:  -15.40390483048238
glc__D_e_1:  -8.052320019492708
glc__D_e_1:  -10.05424014580509
glc__D_e_1:  -7.294480645628386
glc__D_e_1:  -4.879520712220177
glc__D_e_1:  -5.216261700808621
glc__D_e_1:  -1.7748571246129403
glc__D_e_1:  -3.069582534545666
glc__D_e_1:  -1.8358734414106797
glc__D_e_1:  -1.2829730579739596
glc__D_e_1:  -1.308209427704833
glc__D_e_1:  -0.8395478134325108
glc__D_e_1:  -0.323418810161454
glc__D_e_1:  -0.636109278956934
glc__D_e_1:  -0.7074163880933657
glc__D_e_1:  -1.292084723562084
glc__D_e_1:  -1.6325802336387212
glc__D_e_1:  -2.8165987776188706
glc__D_e_1:  -2.4369362745332386
glc__D_e_1:  -2.0954012738505305
glc__D_e_1:  -3.216496521277555
glc__D_e_1:  -1.2135657812407095
glc__D_e_1:  -2.7500372790879495
glc__D_e_1:  -1.636958016856887
glc__D_e_1:  -1.988672368820779
glc__D_e_1:  -0.7506596395556369
glc__D_e_1:  -1.5442981594097438
glc__D_e_1:  -0.3695709877073847
glc__D_e_1:  -0.1144283200924896
strain_number:  0 4.966666666666667
strain_various:  0 4.

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 1,  7,  9,  2, 10,  4, 13,  5, 19, 10, 18,  8, 12,  2, 16, 10,  8,
       11, 10, 17,  6, 16,  4,  3,  7, 11,  5, 15, 16,  6]), 1: array([ 7, 13,  3,  1, 11,  4,  3,  4,  4, 13,  7, 19, 19,  2, 19, 19, 12,
       19, 14,  1,  6,  6, 18,  3, 15,  9,  4, 17,  9,  3])}
1
strain_number:  0 10.9
strain_various:  0 6.090155991434045
strain_number:  1 9.466666666666667
strain_various:  1 6.30202789231812
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 1,  7,  9,  2, 10,  4, 13,  5, 19, 10, 18,  8, 12,  2, 16, 10,  8,
       11, 10, 17,  6, 16,  4,  3,  7, 11,  5, 15, 16,  6]), 1: array([ 7, 13,  3,  1, 11,  4,  3,  4,  4, 13,  7, 19, 19,  2, 19, 19, 12,
       19, 14,  1,  6,  6, 18,  3, 15,  9,  4, 17,  9,  3])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841

glc__D_e_1:  -25.671651257487472
glc__D_e_1:  -1.2046404342798347
glc__D_e_1:  -0.6974601977065977
glc__D_e_1:  -14.91922216096036
glc__D_e_1:  -13.535727684499237
glc__D_e_1:  -1.2119445883846165
glc__D_e_1:  -7.366859088885384
glc__D_e_1:  -15.835488431740137
glc__D_e_1:  -0.4758984038522627
glc__D_e_1:  -10.7189193148675
glc__D_e_1:  -19.12459489970149
glc__D_e_1:  -0.7291979400403594
glc__D_e_1:  -18.87693780518263
glc__D_e_1:  -22.334139989382983
glc__D_e_1:  -0.2342620495160901
glc__D_e_1:  -17.172828219889798
glc__D_e_1:  -6.235398799024118
glc__D_e_1:  -2.3133276255100412
glc__D_e_1:  -5.570847480982762
glc__D_e_1:  -20.523896249913776
glc__D_e_1:  -6.434619326066883
strain_number:  0 42.4
strain_various:  0 21.10071088849852
strain_number:  1 0.4666666666666667
strain_various:  1 0.5617433182117573
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 827.7175932942184, 357.75631714776443, 356.9271802758646, 421.90064650262013, 436.68521338053074, 391.0709155610472, 435.8077232236288, 40

glc__D_e_1:  -30.89517106654686
glc__D_e_1:  -36.746612561357
glc__D_e_1:  -8.751782769419945
glc__D_e_1:  -11.692359705554841
glc__D_e_1:  -7.4483259390255325
glc__D_e_1:  -3.22522882534835
glc__D_e_1:  -2.3936191525712416
glc__D_e_1:  -1.6765327077510552
glc__D_e_1:  -1.1942904462391626
glc__D_e_1:  -0.23896750453479604
glc__D_e_1:  -1.2513546187100084
glc__D_e_1:  -0.3205922688707823
glc__D_e_1:  -0.9011910877509846
glc__D_e_1:  -0.17828622607823752
glc__D_e_1:  -0.11946177091101173
glc__D_e_1:  -1.8783442572916904
glc__D_e_1:  -0.826009639508405
glc__D_e_1:  -2.1327680038796224
glc__D_e_1:  -2.937996861117978
glc__D_e_1:  -1.7479733494741723
glc__D_e_1:  -1.5502918398796357
glc__D_e_1:  -4.581689233086541
glc__D_e_1:  -1.1569192934044341
glc__D_e_1:  -1.0432519335397656
glc__D_e_1:  -0.7963599341524854
strain_number:  0 6.333333333333333
strain_various:  0 10.758975581139477
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 827.7175932942184, 3

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  1,  1, 13,  4, 19,  6, 17,  5, 14, 14, 15, 18, 12,  6, 19,  3,
       10,  4, 17,  2,  9, 17, 12, 13,  7, 19,  6, 13,  8]), 1: array([11,  3,  8, 20,  3, 19, 18, 14, 15, 14,  5,  1, 12,  7, 20, 20,  5,
       19, 10, 16, 19,  1, 16, 18, 16, 20, 12,  3, 19, 17])}
1
strain_number:  0 11.766666666666667
strain_various:  0 6.9889595474258925
strain_number:  1 12.7
strain_various:  1 6.398697784184947
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  1,  1, 13,  4, 19,  6, 17,  5, 14, 14, 15, 18, 12,  6, 19,  3,
       10,  4, 17,  2,  9, 17, 12, 13,  7, 19,  6, 13,  8]), 1: array([11,  3,  8, 20,  3, 19, 18, 14, 15, 14,  5,  1, 12,  7, 20, 20,  5,
       19, 10, 16, 19,  1, 16, 18, 16, 20, 12,  3, 19, 17])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172

glc__D_e_1:  -0.7480401812288449
glc__D_e_1:  -1.1711905338654427
glc__D_e_1:  -22.125697553645352
glc__D_e_1:  -15.806974845525353
glc__D_e_1:  -0.4473848176374773
glc__D_e_1:  -1.0402209579238617
glc__D_e_1:  -12.15456212585786
glc__D_e_1:  -20.038567010815818
glc__D_e_1:  -0.12526658526806145
glc__D_e_1:  -9.363506159686226
glc__D_e_1:  -26.63567936789343
glc__D_e_1:  -3.3254612725292105
glc__D_e_1:  -0.3099113525254491
glc__D_e_1:  -8.316831725139686
glc__D_e_1:  -30.18109695149893
glc__D_e_1:  -4.503745972793965
glc__D_e_1:  -2.4461339159283506
glc__D_e_1:  -13.560475083862348
glc__D_e_1:  -35.397828218060276
glc__D_e_1:  -4.986211472673826
glc__D_e_1:  -2.107107834174939
glc__D_e_1:  -13.739352467995564
glc__D_e_1:  -24.37888030975204
glc__D_e_1:  -3.923913777139468
glc__D_e_1:  -42.338382951624126
glc__D_e_1:  -18.205142418822515
glc__D_e_1:  -1.147093556026407
glc__D_e_1:  -2.900530457342649
glc__D_e_1:  -18.67600281825629
glc__D_e_1:  -23.694432088819074
glc__D_e_1:  -2.751346

glc__D_e_1:  -23.040276007104417
glc__D_e_1:  -23.617426659662232
glc__D_e_1:  -10.87831770908371
glc__D_e_1:  -5.242873972597857
glc__D_e_1:  -2.2963916650674108
glc__D_e_1:  -1.6619649655366953
glc__D_e_1:  -1.1508616893180412
glc__D_e_1:  -2.045159320553794
glc__D_e_1:  -1.8655790347337655
glc__D_e_1:  -1.005701598924086
glc__D_e_1:  -0.5361789502882789
glc__D_e_1:  -1.1796286038692125
glc__D_e_1:  -1.6621852536769577
glc__D_e_1:  -2.7152585807396616
glc__D_e_1:  -0.1033086541744676
glc__D_e_1:  -1.2887606963391995
glc__D_e_1:  -1.283823944716664
glc__D_e_1:  -2.5767735231997864
glc__D_e_1:  -1.5817839116138106
glc__D_e_1:  -2.3500203665080837
glc__D_e_1:  -1.522561078682171
glc__D_e_1:  -1.4214953287549577
glc__D_e_1:  -0.49661741655038827
glc__D_e_1:  -0.48781603284530717
glc__D_e_1:  -0.4326510525451188
glc__D_e_1:  -2.9564522382624188
glc__D_e_1:  -1.1701464875677519
glc__D_e_1:  -1.342766318410808
glc__D_e_1:  -0.3938794974508604
strain_number:  0 9.666666666666666
strain_vario

strain_number:  0 0.06666666666666667
strain_various:  0 0.24944382578492943
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 790.799584187742, 280.0782281444046, 326.65132564803616, 373.67749819839605, 364.83936124283304, 411.638634760243, 430.7580715206141, 344.28366920644476, 378.1378678106534, 368.7159079233967, 335.93120557687246, 304.8111119494212, 364.79369253373125, 361.2735457877557, 419.8216756714081, 250.53221657569483, 265.20986438573766, 234.62332887700535, 282.4765338827839, 227.75598998332546, 203.4297735869744, 109.24285714285715, 20.357142857142854, 46.5, 41.25, 22.5, 15.0]
slip_r:  15.0
29
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 790.799584187742, 280.0782281444046, 326.65132564803616, 373.67749819839605, 364.83936124283304, 411.638634760243, 430.7580715206141, 344.28366920644476, 378.1378678106534, 368.7159079233967, 335.93120557687246, 3

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([17,  9,  9, 13,  4, 10,  5,  5, 11,  5, 19,  1,  5,  1,  4,  5, 17,
       14,  9, 18,  9, 15,  5,  5,  5,  3,  2,  5,  7, 16]), 1: array([ 4, 15,  8, 14,  9,  4, 17, 13, 16, 16, 16, 14,  7,  5,  9,  5,  4,
       14,  4,  2, 10,  4, 18,  1,  7, 17, 16, 19, 20,  4])}
1
strain_number:  0 9.766666666666667
strain_various:  0 6.264627327747083
strain_number:  1 10.4
strain_various:  1 5.771192828754439
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([17,  9,  9, 13,  4, 10,  5,  5, 11,  5, 19,  1,  5,  1,  4,  5, 17,
       14,  9, 18,  9, 15,  5,  5,  5,  3,  2,  5,  7, 16]), 1: array([ 4, 15,  8, 14,  9,  4, 17, 13, 16, 16, 16, 14,  7,  5,  9,  5,  4,
       14,  4,  2, 10,  4, 18,  1,  7, 17, 16, 19, 20,  4])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -10.551364798814593
glc__D_e_1:  -19.842114030146206
glc__D_e_1:  -13.77131885010332
glc__D_e_1:  -12.393891864629165
glc__D_e_1:  -31.198951023186726
glc__D_e_1:  -0.8787626584754236
glc__D_e_1:  -13.028910758182677
glc__D_e_1:  -18.513487870189042
glc__D_e_1:  -14.47382787669003
glc__D_e_1:  -0.002909731733829801
glc__D_e_1:  -9.803069054165704
glc__D_e_1:  -20.23794441307679
glc__D_e_1:  -9.546687412911513
glc__D_e_1:  -11.908910983283688
glc__D_e_1:  -5.293095089997834
strain_number:  0 41.733333333333334
strain_various:  0 18.52373852355104
strain_number:  1 0.7666666666666667
strain_various:  1 0.9893881386437221
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 636.1746992687395, 301.286191749427, 366.61112410551266, 385.411197136197, 444.1796953046954, 484.09869976514716, 448.39300283050284, 405.57475341579755, 440.4513911882333, 445.9297173111689, 381.12495338168316, 310.4103870101658, 320.87652129743685, 303.19409625890864, 382.4915746543119]
slip_r:  382.4915746543119


glc__D_e_1:  -0.625314396074256
glc__D_e_1:  -0.290217427787267
glc__D_e_1:  -0.5506683838678306
glc__D_e_1:  -1.7873506740305618
glc__D_e_1:  -3.3051195509608684
glc__D_e_1:  -2.4902086888246986
glc__D_e_1:  -6.914088198592134
glc__D_e_1:  -7.022511888077559
glc__D_e_1:  -7.740771417451132
glc__D_e_1:  -8.142132768675813
glc__D_e_1:  -1.3371078112054917
glc__D_e_1:  -0.635102033920866
glc__D_e_1:  -1.1628240287753713
glc__D_e_1:  -0.33667915286038963
glc__D_e_1:  -0.3033416285146735
glc__D_e_1:  -1.9187326457227092
glc__D_e_1:  -1.4096164282357584
glc__D_e_1:  -3.9016132464602378
glc__D_e_1:  -2.589725430396949
glc__D_e_1:  -5.3333343730274585
glc__D_e_1:  -5.718595952166577
glc__D_e_1:  -9.469588049516966
glc__D_e_1:  -5.845001037642164
glc__D_e_1:  -11.77698622066847
glc__D_e_1:  -6.460237925409596
strain_number:  0 5.266666666666667
strain_various:  0 4.449219656923622
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 636.1746992687395, 301.286

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 18, 15, 14, 13, 20, 18,  8,  6,  6, 10, 16,  8,  8,  6,  4, 17,
       12, 11,  3,  1,  8,  1, 20, 19, 10,  5,  9, 16, 14]), 1: array([ 3, 20, 13,  8, 18, 18, 12,  4,  9, 17,  9,  3, 20,  9, 11,  7, 19,
       15, 18,  8,  6, 10,  8, 14, 12,  1, 16, 12, 13,  3])}
1
strain_number:  0 12.366666666666667
strain_various:  0 6.853142020676031
strain_number:  1 11.2
strain_various:  1 5.449158956511852
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 18, 15, 14, 13, 20, 18,  8,  6,  6, 10, 16,  8,  8,  6,  4, 17,
       12, 11,  3,  1,  8,  1, 20, 19, 10,  5,  9, 16, 14]), 1: array([ 3, 20, 13,  8, 18, 18, 12,  4,  9, 17,  9,  3, 20,  9, 11,  7, 19,
       15, 18,  8,  6, 10,  8, 14, 12,  1, 16, 12, 13,  3])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -1.610852616146321
glc__D_e_1:  -17.212982831345446
glc__D_e_1:  -0.31365994881021564
glc__D_e_1:  -12.47564821999385
glc__D_e_1:  -1.1683598424386241
glc__D_e_1:  -24.31908628420864
glc__D_e_1:  -15.769072322427236
glc__D_e_1:  -1.0113890412391453
glc__D_e_1:  -20.51968849809483
glc__D_e_1:  -25.197722090441946
glc__D_e_1:  -1.3578501295189547
glc__D_e_1:  -0.2851412926826655
glc__D_e_1:  -11.988240067579063
strain_number:  0 42.233333333333334
strain_various:  0 17.579691565996132
strain_number:  1 2.1
strain_various:  1 1.3503086067019396
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 712.4190267292048, 323.5996556977594, 353.38978017596435, 412.6059981684982, 405.79661018805757, 347.04422689883216, 371.114340103287, 397.3287642966291, 412.0854074609185, 344.3059910089728, 379.84751350286535, 372.3325659959851, 404.1585332139301]
slip_r:  404.1585332139301
15
glc__D_e_1:  -9.838892998939059
glc__D_e_1:  -0.5338141108897467
glc__D_e_1:  -8.363324242574091
glc__D_e_1:  -9.121

glc__D_e_1:  -0.22390095540500804
glc__D_e_1:  -0.09803369077939617
glc__D_e_1:  -0.9034080055386218
glc__D_e_1:  -2.2136803121989397
glc__D_e_1:  -0.936603594416597
glc__D_e_1:  -1.9669145197845053
glc__D_e_1:  -4.0197410395486814
glc__D_e_1:  -1.498102656303296
glc__D_e_1:  -2.9490470754357974
glc__D_e_1:  -2.628447538592352
glc__D_e_1:  -4.561953315574365
glc__D_e_1:  -3.23160107717103
glc__D_e_1:  -7.717159587692999
glc__D_e_1:  -3.2190381481131207
glc__D_e_1:  -6.2198891049476
glc__D_e_1:  -1.5785016888067924
glc__D_e_1:  -2.097438263272667
glc__D_e_1:  -1.8277593895742512
glc__D_e_1:  -1.6155100109431724
glc__D_e_1:  -0.3231851716346569
glc__D_e_1:  -0.8401080016179157
glc__D_e_1:  -0.650806893237182
glc__D_e_1:  -0.2218340334531616
strain_number:  0 3.3666666666666667
strain_various:  0 3.146249972409835
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 712.4190267292048, 323.5996556977594, 353.38978017596435, 412.6059981684982, 405.79661018

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([17,  7,  2,  6, 11,  4,  7,  4,  8, 14,  3, 13,  2, 14,  6,  5, 11,
        6,  7, 14,  6,  9,  7, 18,  5, 13,  8,  7,  3, 13]), 1: array([ 1, 10, 13,  1,  7, 13, 10,  5, 18, 11,  2,  8, 17, 18, 20, 13, 18,
       19, 10, 14, 18, 12, 18, 15, 19,  1,  5, 11,  7,  7])}
1
strain_number:  0 9.533333333333333
strain_various:  0 5.17515431868676
strain_number:  1 11.366666666666667
strain_various:  1 5.879247873287496
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([17,  7,  2,  6, 11,  4,  7,  4,  8, 14,  3, 13,  2, 14,  6,  5, 11,
        6,  7, 14,  6,  9,  7, 18,  5, 13,  8,  7,  3, 13]), 1: array([ 1, 10, 13,  1,  7, 13, 10,  5, 18, 11,  2,  8, 17, 18, 20, 13, 18,
       19, 10, 14, 18, 12, 18, 15, 19,  1,  5, 11,  7,  7])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639

glc__D_e_1:  -6.746703869452617
glc__D_e_1:  -1.1509924540970529
glc__D_e_1:  -16.035534807199518
glc__D_e_1:  -13.598901532977285
glc__D_e_1:  -1.0701306054011361
glc__D_e_1:  -0.34458223307935487
glc__D_e_1:  -7.884907310930799
glc__D_e_1:  -22.07091474738557
glc__D_e_1:  -3.946432404453857
glc__D_e_1:  -1.1919732057606083
glc__D_e_1:  -9.772991707873292
glc__D_e_1:  -23.841669224818027
glc__D_e_1:  -4.865514218188206
glc__D_e_1:  -1.88941911740142
glc__D_e_1:  -7.868704058861009
glc__D_e_1:  -9.451840146160125
glc__D_e_1:  -0.14676125811081242
glc__D_e_1:  -1.4475245643599826
glc__D_e_1:  -18.87443717269319
glc__D_e_1:  -17.97803133142261
glc__D_e_1:  -1.7398929311527134
glc__D_e_1:  -0.3772374830414802
glc__D_e_1:  -4.795482288109211
glc__D_e_1:  -27.8141925670014
glc__D_e_1:  -0.2649531333846582
glc__D_e_1:  -24.663193699934425
glc__D_e_1:  -17.14783234650169
glc__D_e_1:  -34.22696034904848
glc__D_e_1:  -21.2127667082148
glc__D_e_1:  -31.36944882903168
strain_number:  0 39.0333333

glc__D_e_1:  -0.9115170389622593
glc__D_e_1:  -2.724869591266758
glc__D_e_1:  -0.4598338741014033
glc__D_e_1:  -0.2643804038687847
glc__D_e_1:  -0.07174777390101272
glc__D_e_1:  -3.244097652715638
glc__D_e_1:  -0.8199219643019989
glc__D_e_1:  -0.6937089339381657
glc__D_e_1:  -0.7534795653921798
glc__D_e_1:  -1.1280416476785855
glc__D_e_1:  -2.553428191831278
glc__D_e_1:  -1.5598370495086151
glc__D_e_1:  -3.5479580497562173
strain_number:  0 6.333333333333333
strain_various:  0 5.527707983925667
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 713.4453754449884, 418.81599993144107, 347.51023976023964, 448.11426119030614, 467.6303184045058, 411.1174395779659, 416.243908722856, 393.98899974613425, 348.62282129420294, 347.0324702629462, 391.1222355108771, 416.327816565554, 363.3539280438104, 413.80852397525234, 353.3854474367592, 309.1099922811898, 299.9060769613028, 256.16069220420957, 172.81099374481727, 204.20778551158986]
slip_r:  204.207785511589

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 19,  6,  4,  2,  9, 13,  8,  6,  1, 14, 11, 16, 20, 16, 20,  3,
        5, 11, 12,  8, 10,  2, 11, 15,  1, 19, 16,  7, 18]), 1: array([18,  7, 19, 16,  4, 20, 17, 11, 17, 10,  5, 10, 19, 17,  2, 11, 14,
       17,  1,  2, 18,  5,  4,  6, 17,  7,  6, 10, 17,  2])}
1
strain_number:  0 12.166666666666666
strain_various:  0 7.0573051199133765
strain_number:  1 10.966666666666667
strain_various:  1 6.204747286464525
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10, 19,  6,  4,  2,  9, 13,  8,  6,  1, 14, 11, 16, 20, 16, 20,  3,
        5, 11, 12,  8, 10,  2, 11, 15,  1, 19, 16,  7, 18]), 1: array([18,  7, 19, 16,  4, 20, 17, 11, 17, 10,  5, 10, 19, 17,  2, 11, 14,
       17,  1,  2, 18,  5,  4,  6, 17,  7,  6, 10, 17,  2])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693

glc__D_e_1:  -9.279299970385681
glc__D_e_1:  -6.420110424079034
glc__D_e_1:  -11.142627616319405
glc__D_e_1:  -13.13933168350649
glc__D_e_1:  -15.403618891707882
glc__D_e_1:  -9.009972830301905
glc__D_e_1:  -6.839196667183098
glc__D_e_1:  -21.119018193009662
glc__D_e_1:  -7.761730083756743
glc__D_e_1:  -10.271543194575814
glc__D_e_1:  -21.423751696183828
glc__D_e_1:  -31.507471268125066
glc__D_e_1:  -17.274331813979042
glc__D_e_1:  -9.426704740059865
glc__D_e_1:  -7.806549298334275
glc__D_e_1:  -12.414629960302928
glc__D_e_1:  -23.83735706204676
glc__D_e_1:  -23.197258199235495
glc__D_e_1:  -20.815521433877656
glc__D_e_1:  -9.515941875156633
glc__D_e_1:  -12.507751127458084
glc__D_e_1:  -16.477795462491148
glc__D_e_1:  -10.697766685657106
strain_number:  0 38.1
strain_various:  0 21.483869918305373
strain_number:  1 1.1
strain_various:  1 1.8321208111548393
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 996.1335277854238, 365.99887904493175, 404.3209049722207, 504.36045936236934, 562.59330

glc__D_e_1:  -5.3297392882929575
glc__D_e_1:  -4.486135140518498
glc__D_e_1:  -2.377580282645411
glc__D_e_1:  -2.7651157556268515
glc__D_e_1:  -1.6420593627322817
glc__D_e_1:  -1.4387419104007546
glc__D_e_1:  -0.30366620906283565
glc__D_e_1:  -0.32351355260823267
glc__D_e_1:  -0.11374643177933896
strain_number:  0 4.2
strain_various:  0 3.4775470281986602
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 996.1335277854238, 365.99887904493175, 404.3209049722207, 504.36045936236934, 562.5933056827158, 593.5402281734599, 646.0729515160383, 566.1062429571991, 533.2423296525633, 515.7228218545381, 519.8464621556316, 435.1064665323103, 401.1715982020516, 289.2942174793217, 269.98826955955644, 251.47937119887362, 229.0982773785435, 236.57963564213566, 232.63209298871067]
slip_r:  232.63209298871067
21
glc__D_e_1:  -0.4238690214443134
glc__D_e_1:  -2.4621068061052176
glc__D_e_1:  -1.551109204288248
glc__D_e_1:  -0.85390582270

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20, 18,  4,  5,  6,  5, 11, 10,  2,  5,  3,  5,  2,  6, 17, 18, 17,
       12,  9, 15, 14,  1,  1, 16, 14, 17,  8,  9,  1,  7]), 1: array([15, 10,  9, 16, 17, 16,  9, 15, 17, 18, 20, 17, 12,  5, 13, 15, 20,
       11, 15,  3,  4, 11, 15, 13, 16, 10, 12, 13, 13, 16])}
1
strain_number:  0 10.766666666666667
strain_various:  0 7.0931108987680584
strain_number:  1 13.2
strain_various:  1 4.206344414492628
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20, 18,  4,  5,  6,  5, 11, 10,  2,  5,  3,  5,  2,  6, 17, 18, 17,
       12,  9, 15, 14,  1,  1, 16, 14, 17,  8,  9,  1,  7]), 1: array([15, 10,  9, 16, 17, 16,  9, 15, 17, 18, 20, 17, 12,  5, 13, 15, 20,
       11, 15,  3,  4, 11, 15, 13, 16, 10, 12, 13, 13, 16])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49

glc__D_e_1:  -6.841733420363813
glc__D_e_1:  -10.737405336900366
glc__D_e_1:  -21.825053653873734
glc__D_e_1:  -5.52618277426771
glc__D_e_1:  -2.7881661679077867
glc__D_e_1:  -13.850247645546087
glc__D_e_1:  -7.072284700187899
glc__D_e_1:  -11.420283922735049
glc__D_e_1:  -8.553599362379652
glc__D_e_1:  -11.654946953882046
glc__D_e_1:  -19.921481934667778
glc__D_e_1:  -41.84189665353129
glc__D_e_1:  -0.5366871885982011
glc__D_e_1:  -12.760132675625218
glc__D_e_1:  -23.83273581000632
glc__D_e_1:  -17.502079024692208
glc__D_e_1:  -9.7555904365991
strain_number:  0 36.46666666666667
strain_various:  0 15.961899079439833
strain_number:  1 2.2666666666666666
strain_various:  1 2.6699979192667715
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 621.9163372746964, 344.05638163436146, 376.3661558255985, 484.1876567060392, 518.0351884886477, 492.47245219409916, 514.1336021817833, 601.8107422548212, 584.1890137121887, 498.4953955688727, 476.13576968642235, 495.3064562579091, 422.17058388824984, 387.98

glc__D_e_1:  -0.015734996207333873
glc__D_e_1:  -0.3350071841790032
glc__D_e_1:  -0.1007166037562155
glc__D_e_1:  -0.26366863758862724
glc__D_e_1:  -0.8040982396485932
glc__D_e_1:  -3.1037605143872233
glc__D_e_1:  -8.434856735399793
glc__D_e_1:  -13.90131645492588
glc__D_e_1:  -3.499859817869398
glc__D_e_1:  -5.61503195057117
glc__D_e_1:  -0.20231915048120452
glc__D_e_1:  -0.05235887980551146
glc__D_e_1:  -4.470603684873242
glc__D_e_1:  -3.3614795784342557
glc__D_e_1:  -0.8346658237920914
glc__D_e_1:  -0.39230559092666506
glc__D_e_1:  -0.3500077608056478
glc__D_e_1:  -0.005260553530801904
glc__D_e_1:  -0.6128856382808511
glc__D_e_1:  -0.47332463852181217
glc__D_e_1:  -0.010343810091522965
glc__D_e_1:  -1.5816190872951899
glc__D_e_1:  -1.225605404659886
glc__D_e_1:  -2.1949758949189575
glc__D_e_1:  -2.3006274256434383
glc__D_e_1:  -1.9506616499244127
glc__D_e_1:  -2.514572855837884
strain_number:  0 3.5
strain_various:  0 4.08860204307862
strain_number:  1 0.06666666666666667
strain_var

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 17, 19,  5, 18,  9, 12,  2,  1, 10, 18,  7,  6, 16, 12, 18,  1,
        4,  8,  6, 14,  1, 17,  3, 14, 14, 16,  7,  9, 20]), 1: array([ 9, 15,  7, 15, 17, 19, 18,  6,  2, 19, 11, 11, 16,  8, 18,  1, 17,
        5, 16, 18,  4, 18,  5, 12,  7,  1,  7,  7,  9, 14])}
1
strain_number:  0 11.866666666666667
strain_various:  0 7.246761728907303
strain_number:  1 11.066666666666666
strain_various:  1 5.761558431149992
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 17, 19,  5, 18,  9, 12,  2,  1, 10, 18,  7,  6, 16, 12, 18,  1,
        4,  8,  6, 14,  1, 17,  3, 14, 14, 16,  7,  9, 20]), 1: array([ 9, 15,  7, 15, 17, 19, 18,  6,  2, 19, 11, 11, 16,  8, 18,  1, 17,
        5, 16, 18,  4, 18,  5, 12,  7,  1,  7,  7,  9, 14])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -15.9674369612784
glc__D_e_1:  -0.5809712247066123
glc__D_e_1:  -9.08577814681497
glc__D_e_1:  -7.609324037458499
glc__D_e_1:  -11.755416961990424
glc__D_e_1:  -20.31755352027412
glc__D_e_1:  -1.1604955173068379
glc__D_e_1:  -19.628101549901285
glc__D_e_1:  -19.134663461371012
glc__D_e_1:  -1.3761783489704946
glc__D_e_1:  -14.699458507249222
glc__D_e_1:  -11.007792872088622
glc__D_e_1:  -0.8510413203412011
glc__D_e_1:  -14.170154335111377
glc__D_e_1:  -0.5492192571226084
glc__D_e_1:  -6.813823034923356
glc__D_e_1:  -9.355804147726506
glc__D_e_1:  -0.050725259677193435
glc__D_e_1:  -8.135095943188606
glc__D_e_1:  -3.8349733362708207
glc__D_e_1:  -0.6112812967439849
glc__D_e_1:  -4.175863313383779
glc__D_e_1:  -1.6208453222894539
glc__D_e_1:  -2.273945259404641
glc__D_e_1:  -5.639525305383515
glc__D_e_1:  -0.8954865537260597
glc__D_e_1:  -12.103158273184025
glc__D_e_1:  -21.956906229136095
glc__D_e_1:  -28.809980274670135
glc__D_e_1:  -22.246914980730107
glc__D_e_1:  -17.956

glc__D_e_1:  -1.6810871215652081
glc__D_e_1:  -1.2714175980406726
glc__D_e_1:  -1.5929021791264804
glc__D_e_1:  -1.033065641214165
glc__D_e_1:  -4.033764985937187
glc__D_e_1:  -3.044028606320931
glc__D_e_1:  -5.862097310537857
glc__D_e_1:  -4.551663691437478
glc__D_e_1:  -3.914622341926802
glc__D_e_1:  -4.825025094884861
glc__D_e_1:  -4.502734381575606
glc__D_e_1:  -5.550090616720606
glc__D_e_1:  -4.006771627726093
glc__D_e_1:  -2.327620818635654
glc__D_e_1:  -1.5486580590199444
glc__D_e_1:  -0.8961923897321736
glc__D_e_1:  -0.036944600853717535
glc__D_e_1:  -0.6194511849667879
glc__D_e_1:  -0.5470031742198205
strain_number:  0 3.8666666666666667
strain_various:  0 2.15612821717283
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 917.2303511117366, 347.6836668878077, 342.34640841614527, 349.78253654240496, 417.9524574887478, 407.39976742273006, 398.3508747344735, 440.27747511491447, 480.69222037617305, 420.7353388517342, 313.4706241799239, 326.983

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([15,  3,  2, 15,  7, 12, 15,  6, 17,  9, 13, 18, 17, 16, 17,  4, 18,
        8, 20, 19, 13, 18, 11, 16, 11,  8, 10,  7,  4, 16]), 1: array([10, 17,  2, 11,  2,  5,  1,  4, 18, 15,  9, 10, 10,  8, 14,  7, 12,
        7, 18,  2, 17,  3, 13, 18,  2, 15,  7, 20, 19, 16])}
1
strain_number:  0 14.2
strain_various:  0 6.326663154196425
strain_number:  1 10.4
strain_various:  1 5.947548514584251
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([15,  3,  2, 15,  7, 12, 15,  6, 17,  9, 13, 18, 17, 16, 17,  4, 18,
        8, 20, 19, 13, 18, 11, 16, 11,  8, 10,  7,  4, 16]), 1: array([10, 17,  2, 11,  2,  5,  1,  4, 18, 15,  9, 10, 10,  8, 14,  7, 12,
        7, 18,  2, 17,  3, 13, 18,  2, 15,  7, 20, 19, 16])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404

strain_number:  0 44.8
strain_various:  0 16.759872712324917
strain_number:  1 2.0
strain_various:  1 2.160246899469287
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 681.015598436651, 301.1475386069038, 309.244140586352, 395.4705858571127, 365.394901063462, 412.90397706282, 443.18986320443463, 427.1019713085137, 352.29161644260137, 389.91762757019717, 348.5274508802478, 328.0732633550495]
slip_r:  328.0732633550495
14
glc__D_e_1:  -0.771490625393831
glc__D_e_1:  -13.51342025815189
glc__D_e_1:  -21.56462953821684
glc__D_e_1:  -22.77961369760418
glc__D_e_1:  -13.851997915457632
glc__D_e_1:  -29.71198411576562
glc__D_e_1:  -10.383526233811216
glc__D_e_1:  -14.004670383816583
glc__D_e_1:  -21.179315798619214
glc__D_e_1:  -0.5631358772230577
glc__D_e_1:  -0.2404794125056981
glc__D_e_1:  -6.219764353965287
glc__D_e_1:  -17.742185862189874
glc__D_e_1:  -0.8353734134874671
glc__D_e_1:  -0.043491007852282104
glc__D_e_1:  -6.022775949311871
glc__D_e_1:  -15.117309695196344
glc__D_e_1:  -10.08620258

glc__D_e_1:  -6.732777558977204
glc__D_e_1:  -7.863067458180859
glc__D_e_1:  -4.421303895816085
glc__D_e_1:  -8.501130260935215
glc__D_e_1:  -2.6426670962390646
glc__D_e_1:  -3.6215087321200485
glc__D_e_1:  -4.592219268690259
glc__D_e_1:  -2.2484158947308908
glc__D_e_1:  -2.217899352032198
glc__D_e_1:  -1.3756748806135772
glc__D_e_1:  -1.6540804329151775
glc__D_e_1:  -1.0565026148289691
glc__D_e_1:  -1.1648272812127485
glc__D_e_1:  -0.16671051363496925
glc__D_e_1:  -0.13383899222033135
glc__D_e_1:  -0.22118616918756873
glc__D_e_1:  -0.3229534810067508
glc__D_e_1:  -1.0863229021776277
glc__D_e_1:  -0.39896451908120634
glc__D_e_1:  -1.071259414696136
glc__D_e_1:  -1.8043242937240178
glc__D_e_1:  -0.3729269250396864
glc__D_e_1:  -1.044496409271556
glc__D_e_1:  -3.3133740993807
glc__D_e_1:  -0.8990519795061068
glc__D_e_1:  -2.8648676382672984
glc__D_e_1:  -1.2679616566875245
strain_number:  0 3.9
strain_various:  0 3.0259984578096977
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_thresh

In [12]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 18, 16,  8, 12, 13, 15, 12,  2, 16,  5, 20,  7, 11, 12,  6, 14,
        9, 13,  2,  2, 17,  2, 18, 20,  6, 19, 13,  9,  5]), 1: array([12,  6,  5,  3, 15,  4, 14, 13,  4, 13, 10,  6,  4, 17,  8,  5, 10,
        7, 10, 14, 20,  7, 15, 18,  3, 17, 17, 10,  5,  6])}
1
strain_number:  0 12.6
strain_various:  0 6.902173570694958
strain_number:  1 9.933333333333334
strain_various:  1 5.0526120857323775
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 18, 16,  8, 12, 13, 15, 12,  2, 16,  5, 20,  7, 11, 12,  6, 14,
        9, 13,  2,  2, 17,  2, 18, 20,  6, 19, 13,  9,  5]), 1: array([12,  6,  5,  3, 15,  4, 14, 13,  4, 13, 10,  6,  4, 17,  8,  5, 10,
        7, 10, 14, 20,  7, 15, 18,  3, 17, 17, 10,  5,  6])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -2.876821875964307
glc__D_e_1:  -15.446341681845542
glc__D_e_1:  -27.130236986557357
glc__D_e_1:  -0.43267021663872285
glc__D_e_1:  -0.053964766106519235
glc__D_e_1:  -12.277410253133537
glc__D_e_1:  -29.13348102491001
glc__D_e_1:  -31.191551241069238
glc__D_e_1:  -23.351342144568676
glc__D_e_1:  -0.36314284734379143
glc__D_e_1:  -0.2957221873138316
glc__D_e_1:  -33.33303615956561
strain_number:  0 43.53333333333333
strain_various:  0 16.097066675501953
strain_number:  1 1.3666666666666667
strain_various:  1 1.53803626600791
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 682.0510211537686, 390.5823293708354, 414.4332322089676, 389.97046046059205, 421.24994888152787, 500.32905454760305, 459.4869774215448, 357.5460376885782, 389.32167337469014, 434.74035891953116, 388.5230021653447, 282.8421541066175, 412.62026476229397]
slip_r:  412.62026476229397
15
glc__D_e_1:  -29.146860354081497
glc__D_e_1:  -15.905152629685563
glc__D_e_1:  -19.446051665796126
glc__D_e_1:  -15.8427441038087

glc__D_e_1:  -1.7526014073678504
glc__D_e_1:  -0.7275983937131518
glc__D_e_1:  -2.4054804121700766
glc__D_e_1:  -2.3065656518259816
glc__D_e_1:  -2.4094179996644383
glc__D_e_1:  -0.6597824801525038
glc__D_e_1:  -0.56859856842238
glc__D_e_1:  -1.5045435106573564
glc__D_e_1:  -0.7863779111069564
glc__D_e_1:  -0.9456296198865002
glc__D_e_1:  -0.5756687377732858
glc__D_e_1:  -0.15325259244874379
glc__D_e_1:  -1.1549275112896629
glc__D_e_1:  -1.9440422692719674
glc__D_e_1:  -2.428088844680033
glc__D_e_1:  -1.0736496640902735
glc__D_e_1:  -2.6044427031020176
glc__D_e_1:  -1.4506590234896435
glc__D_e_1:  -3.1327648254734064
glc__D_e_1:  -1.7559267875690896
glc__D_e_1:  -0.6496010153029715
glc__D_e_1:  -1.7227617378883595
glc__D_e_1:  -0.43606492506742933
glc__D_e_1:  -1.1332608824964354
glc__D_e_1:  -0.16201010411949457
glc__D_e_1:  -1.3473683948059068
glc__D_e_1:  -1.31156521235929
glc__D_e_1:  -0.3783879892571623
glc__D_e_1:  -1.4806516931777303
strain_number:  0 2.5
strain_various:  0 1.33

In [13]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6,  7, 11,  4, 11,  3,  1,  9,  6,  8, 14, 18,  7,  4, 13, 12, 20,
       12, 13,  4, 17, 15,  7,  9,  9, 13,  8, 19, 10, 18]), 1: array([18, 10, 12,  2,  4, 13, 17, 19,  3,  9,  6,  3, 16,  8,  6, 15,  7,
        4, 11, 11,  5,  7,  2, 18, 19, 17, 20,  8,  8,  9])}
1
strain_number:  0 11.833333333333334
strain_various:  0 6.000462945103116
strain_number:  1 10.233333333333333
strain_various:  1 5.6254382545323125
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 6,  7, 11,  4, 11,  3,  1,  9,  6,  8, 14, 18,  7,  4, 13, 12, 20,
       12, 13,  4, 17, 15,  7,  9,  9, 13,  8, 19, 10, 18]), 1: array([18, 10, 12,  2,  4, 13, 17, 19,  3,  9,  6,  3, 16,  8,  6, 15,  7,
        4, 11, 11,  5,  7,  2, 18, 19, 17, 20,  8,  8,  9])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693

glc__D_e_1:  -25.253557920670016
glc__D_e_1:  -2.082360008179535
glc__D_e_1:  -0.1532966484844419
glc__D_e_1:  -6.132581589944031
strain_number:  0 39.6
strain_various:  0 16.055321028660043
strain_number:  1 1.4333333333333333
strain_various:  1 1.2827920936595902
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 614.3399986881025, 408.44444371418064, 388.94835011479756, 469.5731178908809, 529.464486779491, 477.1236039827869, 398.268963724951, 413.78795423338784, 454.1676014759565, 419.41646289265395, 375.2725280494476, 308.66085377880927, 348.6048711121715]
slip_r:  348.6048711121715
15
glc__D_e_1:  -16.43535967063123
glc__D_e_1:  -36.461431453094114
glc__D_e_1:  -18.987646083659182
glc__D_e_1:  -17.207537682754616
glc__D_e_1:  -16.538437525031586
glc__D_e_1:  -23.20666072998388
glc__D_e_1:  -17.583203171773132
glc__D_e_1:  -19.34663321306005
glc__D_e_1:  -14.359393689938619
glc__D_e_1:  -23.61418876776186
glc__D_e_1:  -18.972222866846245
glc__D_e_1:  -0.5450637060132166
glc__D_e_1:  -19.49

glc__D_e_1:  -5.930991884356635
glc__D_e_1:  -14.529923544712785
glc__D_e_1:  -9.15377238970003
glc__D_e_1:  -12.351110254656312
glc__D_e_1:  -9.373748666154789
glc__D_e_1:  -12.143784958017484
glc__D_e_1:  -6.576877488489923
glc__D_e_1:  -4.904726678792393
glc__D_e_1:  -2.0083999741182543
glc__D_e_1:  -1.5092579015622414
glc__D_e_1:  -1.4824815739915411
glc__D_e_1:  -1.7710913032811297
glc__D_e_1:  -0.9158867694287081
glc__D_e_1:  -1.2211776412275932
glc__D_e_1:  -0.19122291371858113
glc__D_e_1:  -0.4649988351367549
glc__D_e_1:  -0.1914023476082234
glc__D_e_1:  -0.0750697598138117
glc__D_e_1:  -0.18451537710844013
glc__D_e_1:  -0.4917452181020251
glc__D_e_1:  -0.18389486766344265
glc__D_e_1:  -0.3842565823154318
glc__D_e_1:  -0.25336848549183233
strain_number:  0 4.233333333333333
strain_various:  0 5.339683719306063
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 614.3399986881025, 408.44444371418064, 388.94835011479756, 469.5731178908809, 529.

In [14]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([14, 12,  1,  3, 11,  8, 16,  2, 13,  9, 11, 13, 10,  5, 20, 20,  6,
        9,  7,  1, 18, 20,  2,  5,  8,  8, 14,  1,  6,  1]), 1: array([ 4, 19,  9, 14,  2,  4, 18, 14, 14, 13,  4,  4, 17, 18,  4, 13, 15,
       13,  3, 12, 10,  3, 19, 16, 12, 16, 15,  5, 20, 20])}
1
strain_number:  0 10.6
strain_various:  0 7.064465065853654
strain_number:  1 11.666666666666666
strain_various:  1 5.855671514770008
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([14, 12,  1,  3, 11,  8, 16,  2, 13,  9, 11, 13, 10,  5, 20, 20,  6,
        9,  7,  1, 18, 20,  2,  5,  8,  8, 14,  1,  6,  1]), 1: array([ 4, 19,  9, 14,  2,  4, 18, 14, 14, 13,  4,  4, 17, 18,  4, 13, 15,
       13,  3, 12, 10,  3, 19, 16, 12, 16, 15,  5, 20, 20])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -10.68094375960223
glc__D_e_1:  -18.020354613200148
glc__D_e_1:  -28.471419746686713
glc__D_e_1:  -19.392467900433193
glc__D_e_1:  -0.29663469116765384
glc__D_e_1:  -17.96203273972296
glc__D_e_1:  -13.492643900992427
glc__D_e_1:  -1.1468715886818757
glc__D_e_1:  -1.1519057839256988
strain_number:  0 39.5
strain_various:  0 16.83399338639924
strain_number:  1 2.1666666666666665
strain_various:  1 2.6967058093574505
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 880.6482240669546, 322.1764386928861, 368.3728234484813, 459.90438675101996, 510.57635384800074, 458.44663494400334, 448.394218591587, 452.7675942041105, 404.13106297003173, 400.32483608230524, 433.96875246764154, 435.47283625522437, 411.66154309036784, 346.186431710943]
slip_r:  346.186431710943
16
glc__D_e_1:  -13.328607428010962
glc__D_e_1:  -0.9828351157004107
glc__D_e_1:  -23.035857661364687
glc__D_e_1:  -15.993694021858571
glc__D_e_1:  -0.6072282852867836
glc__D_e_1:  -27.108539101097584
glc__D_e_1:  -31.1039542542

glc__D_e_1:  -1.3511170569859894
glc__D_e_1:  -3.407530228481944
glc__D_e_1:  -4.031336389072995
glc__D_e_1:  -3.110129054919417
glc__D_e_1:  -2.4002427399383057
glc__D_e_1:  -1.2758453414583144
glc__D_e_1:  -1.4056889986856052
glc__D_e_1:  -0.44109672629743535
glc__D_e_1:  -0.16060220080326215
glc__D_e_1:  -0.44594389613664953
glc__D_e_1:  -0.30264631347383797
glc__D_e_1:  -0.9000634379304668
glc__D_e_1:  -0.07021516896132857
glc__D_e_1:  -0.39596294434072365
glc__D_e_1:  -2.301644793795904
glc__D_e_1:  -0.98602916587087
glc__D_e_1:  -1.0587850816240079
glc__D_e_1:  -0.36891802361920945
glc__D_e_1:  -1.640996581690818
glc__D_e_1:  -1.8385459509190056
glc__D_e_1:  -1.5072929086821374
glc__D_e_1:  -1.3446829171146062
glc__D_e_1:  -10.811116314173237
glc__D_e_1:  -2.925215049750783
glc__D_e_1:  -14.75665647650318
glc__D_e_1:  -4.233907694224563
glc__D_e_1:  -8.712188417794803
strain_number:  0 4.2
strain_various:  0 5.173006862551025
strain_number:  1 0.26666666666666666
strain_various: 